# Random Forest

In [1]:
!pip install imblearn

You should consider upgrading via the '/home/ec2-user/anaconda3/envs/python3/bin/python -m pip install --upgrade pip' command.


In [ ]:
import pandas as pd
import numpy as np

import matplotlib.pyplot as plt
import seaborn as sns
%matplotlib inline

from sklearn.model_selection import train_test_split, GridSearchCV
from sklearn.metrics import classification_report, confusion_matrix, roc_auc_score
from sklearn.metrics import recall_score, precision_score
from sklearn.ensemble import RandomForestClassifier
from sklearn.preprocessing import MinMaxScaler, StandardScaler
from sklearn.inspection import permutation_importance

from imblearn.over_sampling import RandomOverSampler, SMOTE

from src import rf_utils

## UDF

In [3]:
def create_sorted_permutation_importance_df(model, X, y, n_iterations, random_state):
    '''
    Inputs:
    Enter a fitted tree-based model, e.g. Random Forest or Gradient Boost.
    X and y can be the training set or a hold-out set, i.e., validation or test.
    X and y must be dataframes.
    
    Return:
    Returns a sorted dataframe comprised of each feature's importance for the number of selected iterations.
    The dataframe is sorted by the features' medians.
    '''
    
    results = permutation_importance(model, X, y, n_repeats=n_iterations,
                                     random_state=random_state, n_jobs=-1)
    
    df = pd.DataFrame(results.importances.T * 100, columns=X.columns)
    sorted_idx = df.median().sort_values().index[::-1]
    sorted_df = df[sorted_idx]
    return sorted_df


def plot_horizontal_permutation_importance_boxplot(df, figsize, max_features):
    '''
    df must be sorted.
    max_features is a limit on the number of features displayed
    '''
    
    df2 = df.iloc[:, :max_features].copy()
    
    fig, ax = plt.subplots(figsize=figsize)
    
    sns.boxplot(data=df2, orient='h', ax=ax, palette='mako')
    ax.set_xlabel('Importance')
    ax.set_title('Permutation Importance', fontsize=18)
    fig.tight_layout();


def calculate_and_plot_permutation_importance(model, X, y, n_iterations, random_state, figsize, max_features):
    '''
    Inputs:
    Enter a fitted tree-based model, e.g. Random Forest or Gradient Boost.
    X and y can be the training set or a hold-out set, i.e., validation or test.
    X and y must be dataframes.
    max_features is the number of features to be displayed on the boxplot.
    
    Returns a horizontal box plot summarizing each feature's permutation importance.
    '''
    sorted_df = create_sorted_permutation_importance_df(model, X, y, n_iterations, random_state)
    plot_horizontal_permutation_importance_boxplot(sorted_df, figsize, max_features)

## Import Data

In [4]:
accidents = pd.read_csv('data/accidents.csv', index_col=0)
accidents.head()

,RD_NO,CRASH_DATE,POSTED_SPEED_LIMIT,TRAFFIC_CONTROL_DEVICE,DEVICE_CONDITION,WEATHER_CONDITION,LIGHTING_CONDITION,FIRST_CRASH_TYPE,TRAFFICWAY_TYPE,LANE_CNT,...,was_dark_no_lighting,is_day,is_evening,is_morning,is_night,season,is_fall,is_spring,is_summer,is_winter
0,JC343143,07/10/2019 05:56:00 PM,35,NO CONTROLS,NO CONTROLS,CLEAR,DAYLIGHT,TURNING,ONE-WAY,NaN,...,0,1,0,0,0,summer,0,0,1,0
1,JC194776,03/21/2019 10:50:00 PM,30,TRAFFIC SIGNAL,FUNCTIONING PROPERLY,CLEAR,"DARKNESS, LIGHTED ROAD",TURNING,NOT DIVIDED,4.0,...,0,0,0,0,1,spring,0,1,0,0
2,JB200478,03/26/2018 02:23:00 PM,35,NO CONTROLS,NO CONTROLS,CLEAR,DAYLIGHT,PARKED MOTOR VEHICLE,NOT DIVIDED,NaN,...,0,1,0,0,0,spring,0,1,0,0
3,JB415436,08/30/2018 05:45:00 PM,30,TRAFFIC SIGNAL,FUNCTIONING PROPERLY,CLEAR,DAYLIGHT,TURNING,NOT DIVIDED,NaN,...,0,1,0,0,0,summer,0,0,1,0
4,JC301403,06/11/2019 08:40:00 AM,30,TRAFFIC SIGNAL,FUNCTIONING PROPERLY,CLEAR,DAYLIGHT,REAR END,DIVIDED - W/MEDIAN BARRIER,NaN,...,0,0,0,1,0,summer,0,0,1,0


## Process Data for Model

In [8]:
X = accidents[['crash_month',
              'CRASH_HOUR',
              'is_weekend',
              'has_snow_ice',
              'involved_pedest_cycle',
              'at_intersection',
              'at_stop_sign_or_traffic_light',
              'is_day',
              'is_evening',
              'is_morning',
              'is_night',
               'is_rain',
               'is_sleet_snow',
               'is_spring',
               'is_summer',
               'is_weekend',
               'is_winter']]


y = accidents['is_fatal_or_incap']

X_train, X_test, y_train, y_test = train_test_split(X, y, test_size=0.20, random_state=3)

In [15]:
oversampler = RandomOverSampler(sampling_strategy=0.6, random_state=3)
#oversampler = SMOTE(sampling_strategy=0.5, random_state=3)

X_over, y_over = oversampler.fit_resample(X_train.to_numpy(), y_train.to_numpy())

## Hyperparameter Tuning

In [16]:
n_features = X.shape[1]

n_estimators = [10, 100, 500, 700, 1000]
max_features = [2, int(np.sqrt(n_features)), n_features // 2, int(n_features * 0.75), n_features]
max_depth = [2, 3, 4]
params_grid = dict(n_estimators=n_estimators,
                   max_features=max_features,
                   max_depth=max_depth)

In [17]:
rf = RandomForestClassifier()

clf = GridSearchCV(rf, params_grid, cv=5, verbose=0, scoring='precision')

best_model = clf.fit(X_over, y_over)

KeyboardInterrupt: 

In [ ]:
print('Optimal # of Estimators: ', best_model.best_estimator_.get_params()['n_estimators'])
print('Optimal # of Features per Node: ', best_model.best_estimator_.get_params()['max_features'])
print('Max. Depth of Tree: ', best_model.best_estimator_.get_params()['max_depth'])

### Train and Predict

In [ ]:
n_estimators = 
max_features =
max_depth = 

rf = RandomForestClassifier(n_estimators=n_estimators, max_features=max_features, max_depth=max_depth, random_state=3)
rf.train(X_over, y_over)
y_pred = rf.predict(X_test)
y_prob = rf.predict_proba(X_test)[:, 1]


cf_matrix = confusion_matrix(y_test, y_pred)
print(cf_matrix)
print('\n')
print('Recall (Scores FN): ', round(recall_score(y_test, y_pred), 4))
print('Precision (Scores FP): ', round(precision_score(y_test, y_pred), 4))
print('AUC Score: ', round(roc_auc_score(y_test, y_prob), 4))

In [ ]:
plt.figure(figsize=(7,4))
sns.heatmap(cf_matrix, cmap='coolwarm', annot=True, annot_kws={'size': 20, 'fontweight':'bold'}, fmt='d',
                 xticklabels=False, yticklabels=False);
#plt.savefig('img/cf_matrix.png', bbox_inches='tight')

### Permutation Importance

In [ ]:
calculate_and_plot_permutation_importance(rf, X_over, y_over, 10, 3, (10,15), 20)
